In [1]:
# evaluate a persistence forecast model
from pandas import read_csv
import datetime
from pandas import DataFrame, Series
from pandas import concat, merge
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error
from math import sqrt
from numpy import concatenate, savetxt, unique, array, subtract
from statistics import mean


In [2]:
# make a prediction using data from a single meter
# load dataset
dataset = read_csv('CharlestownAllWithDate3.csv', header=0, sep='[,]', parse_dates=True, squeeze=True, dayfirst=True, engine='python') 
#dataset = dataset.loc[dataset['MeterNo'] <3]
meter_ids = unique(dataset['MeterNo'])
print(meter_ids)
df2 = DataFrame(index =['Date'], columns=['prediction', 'actual', 'error','mse'])

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  17  18  19
  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  37  38
  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56
  57  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75
  76  77  78  79  80  81  82  83  84  85  86  88  89  90  91  92  93  94
  95  96  97  98  99 100 101 102 103 104 105 106 107 109 110 113 114 115
 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133
 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151
 152 153 154 155 156 157 159 160 161 162 163 164 165 166 167 168 169 170
 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188
 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206
 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224
 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242
 243 244 245 246 247 248 249 250 251 252 253 254 25

In [3]:
for m in meter_ids:
    series = read_csv('CharlestownAllWithDate3.csv', header=0, sep='[,]', parse_dates=True, squeeze=True, dayfirst=True, engine='python') 
    series = series.set_index(['Date'])
    series = series.loc[series['MeterNo'] == m]
    series.drop(series.columns[[1]], axis = 1, inplace = True)
    #dataset = dataset[['kWh', 'Temp', 'Area_m2', 'Month', 'Weekday', 'Holiday']]
    
    # create lagged dataset
    values = DataFrame(series.values)
    dataframe = concat([values.shift(1), values], axis=1)
    dataframe.columns = ['t', 't+1']
    #print(dataframe.head(5))
    # split into train and test sets
    X = dataframe.values
    train_size = int(len(X) * 0.66)
    train, test = X[1:train_size], X[train_size:]
    train_X, train_y = train[:,0], train[:,1]
    test_X, test_y = test[:,0], test[:,1]
    # persistence model
    def model_persistence(x):
        return x
    # walk-forward validation
    predictions = list()
    for x in test_X:
        yhat = model_persistence(x)
        predictions.append(yhat)
        
    # calculate actual Error
    prediction = mean(predictions)
    actual = mean(test_y)
    mse = mean_squared_error(test_y, predictions)
    rmse = sqrt(mean_squared_error(test_y, predictions))
    error = mean(subtract(test_y, predictions))
    
    test_y = DataFrame(test_y, columns = ['kWh'])
    predictions = DataFrame(predictions, columns = ['kWh']) 
        
    #print('MeterNo %s - Prediction: %.3f - Actual: %.3f - Error: %.3f - Test MSE: %.3f - Test RMSE: %.3f' % (m, prediction, actual, error, mse, rmse))
    df1 = merge(predictions, test_y, left_index=True, right_index=True)
    df1.columns =['prediction', 'actual']
    df1['error'] = df1['prediction'] - df1['actual']
    df1['mse'] = df1['error'] ** 2
    df1['rmse'] = abs(df1['error'])
    #print(df1)
    df2 = df2.append(df1)
    

In [4]:
print(df2)

      prediction  actual  error    mse  rmse
Date         NaN     NaN    NaN    NaN   NaN
0            4.0     5.0   -1.0    1.0   1.0
1            5.0     4.0    1.0    1.0   1.0
2            4.0     5.0   -1.0    1.0   1.0
3            5.0    11.0   -6.0   36.0   6.0
...          ...     ...    ...    ...   ...
111         13.0    25.0  -12.0  144.0  12.0
112         25.0    22.0    3.0    9.0   3.0
113         22.0    23.0   -1.0    1.0   1.0
114         23.0    21.0    2.0    4.0   2.0
115         21.0    23.0   -2.0    4.0   2.0

[31901 rows x 5 columns]


In [5]:
def Summary(x):
    return Series(index=['min','max', 'mean','sum','count'],data=[x.min(),x.max(),x.mean(),x.sum(),x.count()])
df2.apply(Summary)

,prediction,actual,error,mse,rmse
min,0.000000,0.00000,-70.000000,0.000000e+00,0.000000
max,103.000000,103.00000,55.000000,4.900000e+03,70.000000
mean,17.550408,17.54768,0.002727,5.561150e+01,5.036834
sum,559858.000000,559771.00000,87.000000,1.774007e+06,160675.000000
count,31900.000000,31900.00000,31900.000000,3.190000e+04,31900.000000


In [6]:
df2.drop(df2.columns[[2,3,4]], axis = 1, inplace = True)
df3 = df2.groupby(df2.index)['prediction','actual'].sum()
df3['error'] = round(((df3['prediction'] - df3['actual'])),2)
df3['mse'] = df3['error'] ** 2
df3['rmse'] = abs(df3['error'])
df3['%error'] = (df3['error']/df3['actual'])*100
print(df3)
df3.to_csv('CHL_SARIMA_CLH_Persistence_Forecast_Models_ByDate.csv')

      prediction  actual  error       mse   rmse     %error
0         4131.0  4446.0 -315.0   99225.0  315.0  -7.085020
1         4446.0  4196.0  250.0   62500.0  250.0   5.958055
2         4196.0  3790.0  406.0  164836.0  406.0  10.712401
3         3790.0  4570.0 -780.0  608400.0  780.0 -17.067834
4         4570.0  4545.0   25.0     625.0   25.0   0.550055
...          ...     ...    ...       ...    ...        ...
112       4066.0  4075.0   -9.0      81.0    9.0  -0.220859
113       4075.0  3925.0  150.0   22500.0  150.0   3.821656
114       3925.0  3719.0  206.0   42436.0  206.0   5.539123
115       3719.0  4044.0 -325.0  105625.0  325.0  -8.036597
Date         0.0     0.0    0.0       0.0    0.0        NaN

[117 rows x 6 columns]


<ipython-input-6-92b18c512e39>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df3 = df2.groupby(df2.index)['prediction','actual'].sum()


In [7]:
df3.apply(Summary)

,prediction,actual,error,mse,rmse,%error
min,0.000000,0.000000,-1644.00000,0.000000e+00,0.000000,-24.841342
max,8628.000000,8628.000000,1797.00000,3.229209e+06,1797.000000,30.263158
mean,4785.111111,4784.367521,0.74359,2.105611e+05,333.358974,0.406374
sum,559858.000000,559771.000000,87.00000,2.463565e+07,39003.000000,47.139408
count,117.000000,117.000000,117.00000,1.170000e+02,117.000000,116.000000


In [8]:
df4 = df3.mean()
df4

prediction      4785.111111
actual          4784.367521
error              0.743590
mse           210561.136752
rmse             333.358974
%error             0.406374
dtype: float64